In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Modelleri yükle
face_model = load_model('face.h5')
phone_model = load_model('phone.h5')

# Görüntüyü işlemek için fonksiyon
def preprocess_image(image):
    normalized_image = image / 255.0  # Görüntüyü normalize etme
    resized_image = cv2.resize(normalized_image, (120, 120))  # Görüntüyü yeniden boyutlandırma
    return resized_image

# Dikdörtgeni çizmek ve etiketi yazmak için fonksiyon
def draw_rectangle_and_label(frame, coords, label):
    (x1, y1), (x2, y2) = coords
    # Dikdörtgeni çiz
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
    # Etiket dikdörtgenini çiz
    cv2.rectangle(frame, (x1, y1-30), (x1+80, y1), (255, 0, 0), -1)
    # Etiketi yaz
    cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

# Video yakalama
cap = cv2.VideoCapture(0)  # 0, yerleşik kamerayı temsil eder. Eğer farklı bir kamera kullanıyorsanız, uygun indeksi belirtmelisiniz.

while True:
    ret, frame = cap.read()  # Video akışından bir kare oku

    if not ret:
        break

    frame = frame[50:500, 50:500, :]  # Kareyi yeniden boyutlandır

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120, 120))
    
    # Yüz modeliyle tahmin yap
    face_yhat = face_model.predict(np.expand_dims(resized / 255, 0))
    face_coords = face_yhat[1][0]
    
    # Telefon modeliyle tahmin yap
    phone_yhat = phone_model.predict(np.expand_dims(resized / 255, 0))
    phone_coords = phone_yhat[1][0]
    
    if face_yhat[0] > 0.8:
        draw_rectangle_and_label(frame, ((int(face_coords[0]*450), int(face_coords[1]*450)), (int(face_coords[2]*450), int(face_coords[3]*450))), 'Face')
    
    if phone_yhat[0] > 0.8:
        draw_rectangle_and_label(frame, ((int(phone_coords[0]*450), int(phone_coords[1]*450)), (int(phone_coords[2]*450), int(phone_coords[3]*450))), 'Phone')
    
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kaynakları serbest bırak
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 45ms/step
